In [ ]:
import numpy as np
import sounddevice as sd
from IPython.display import Audio

sr = 48000         # sample rate

def white_noise(duration=100):
    # duration in seconds
    dur = duration
    rms_db = -35       # start quiet; raise toward -25 dB if needed
    amp = 10 ** (rms_db / 20)

    mono = np.random.normal(0, amp, int(sr * dur)).astype(np.float32)
    stereo = np.column_stack([mono, mono])  # duplicate to L/R
    return mono, stereo

## play sound to speakers 
# sd.play(stereo, sr, blocking=True)      # plays to default output (your headphones)

In [ ]:
def pink_noise(duration):
    n = duration * sr
    # 1/f pink via frequency shaping
    N = int(2**np.ceil(np.log2(n)))
    freqs = np.fft.rfftfreq(N, 1/48000)
    x = np.random.normal(size=freqs.size) + 1j*np.random.normal(size=freqs.size)
    x[0] = 0
    x /= np.maximum(freqs, 1e-12)      # ~1/f
    y = np.fft.irfft(x, n=N)
    y = y[:n]
    y = y / np.max(np.abs(y)) * 0.05   # set level ≈ -26 dBFS
    return y.astype(np.float32)


In [ ]:
# play sound to speakers on demand:
mono, stereo = white_noise(100)
Audio([mono, mono], rate=sr)

In [ ]:
# save to file
# [to be added]

In [ ]:
def pink_noise(duration):
    n = duration * sr
    # 1/f pink via frequency shaping
    N = int(2**np.ceil(np.log2(n)))
    freqs = np.fft.rfftfreq(N, 1/sr)
    x = np.random.normal(size=freqs.size) + 1j*np.random.normal(size=freqs.size)
    x[0] = 0
    x /= np.maximum(freqs, 1e-12)      # ~1/f
    y = np.fft.irfft(x, n=N)
    y = y[:n]
    y = y / np.max(np.abs(y)) * 0.05   # set level ≈ -26 dBFS
    return y.astype(np.float32)

duration = 100  # seconds
mono = pink_noise(duration)
stereo = np.column_stack([mono, mono])
Audio([mono, mono], rate=sr)

In [ ]:
def octave_noise(dur, octaves=8):
    n = sr * dur  #  dur in seconds
    signal = np.zeros(int(sr * dur), float)
    base_freq = 40  # Hz, approx low e on bass
    ratio = 2  # chose 2 for octave
    ts = np.linspace(0, dur, n)
    for oct in range(0, octaves):
        freq = base_freq * ratio**oct
        print(f"{freq=}")
        signal += np.sin(2* np.pi * ts * freq)

    return ts, signal


duration = 100  # seconds
ts, mono = octave_noise(duration)
Audio(mono, rate=sr)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ts[0:100], mono[0:100])